In [69]:
import pandas as pd
import numpy as np

In [70]:
df = pd.read_csv('sa_to_model_ib.csv')

In [71]:
model = pd.DataFrame(np.zeros([5750,40]))
model.columns = list(range(0,400,10))

for i in range(0,400,10):
    for j in range(0,10):
        model[i] =  model[i] + df[str(i+j)]
model['label'] = df['label']
model['ip'] = df['ip_id']
model.insert(loc=0,column='ip_id',value=model.ip,allow_duplicates=True)
model.drop('ip',axis=1,inplace=True)
#model.drop(model[model[0] == model[0].max()].index,inplace=True)

In [72]:
X = model.iloc[:,1:-1]
y = model.iloc[:,-1]

In [100]:
# tuning XGboost model
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train,y_train)
y_pred_xgb_train = xgb.predict(X_train)
y_pred_xgb_test = xgb.predict(X_test)
print('XGB(train):' ,accuracy_score(y_train, y_pred_xgb_train))
print('XGB(test):' ,accuracy_score(y_test, y_pred_xgb_test))
print(classification_report(y_test, y_pred_xgb_test))

XGB(train): 0.88347826087
XGB(test): 0.846956521739
             precision    recall  f1-score   support

          0       0.87      0.85      0.86       944
          1       0.83      0.84      0.83       781

avg / total       0.85      0.85      0.85      1725



In [5]:
# tuning logistic regression and decision tree model
from sklearn.preprocessing import StandardScaler, MaxAbsScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, 
                     test_size=0.30,
                     stratify=y,
                     random_state=1)

pipe_lr = make_pipeline(StandardScaler(),
                        LogisticRegression(random_state=1))

param_grid_lr = [{'logisticregression__C': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]}]

gs_lr = GridSearchCV(estimator=pipe_lr, 
                  param_grid=param_grid_lr, 
                  scoring='accuracy', 
                  cv=10,
                  refit=True)

gs_lr.fit(X_train, y_train)

y_pred_lr_train = gs_lr.predict(X_train)
y_pred_lr_test = gs_lr.predict(X_test)
# ==========================================================================
pipe_dt = make_pipeline(DecisionTreeClassifier(random_state=1))

param_grid_dt = [{'decisiontreeclassifier__max_depth': [1, 2, 3, 4, 5, 6, 7, None], 
               'decisiontreeclassifier__min_samples_leaf': [1, 2, 3, 4, 5, 6, 7]}]

gs_dt = GridSearchCV(estimator=pipe_dt,
                  param_grid=param_grid_dt,
                  scoring='accuracy',
                  cv=10,
                  refit=True)

gs_dt = gs_dt.fit(X_train, y_train)

y_pred_dt_train = gs_dt.predict(X_train)
y_pred_dt_test = gs_dt.predict(X_test)

In [14]:
# finding logistic regression parameters
lr = gs_lr.best_estimator_.named_steps['logisticregression']
lr.coef_

array([[-0.04693163, -0.24641083,  0.35832458,  0.47681398,  0.42373008,
        -0.05276948, -0.80975095, -0.75787479,  0.01652577, -0.05121102,
        -0.08490939, -1.97480778, -0.84978966,  0.29547   ,  0.49625642,
         0.7818643 , -0.01737768, -0.13293516, -0.17063149, -0.08278076,
         0.50758553, -0.41726046, -0.27637697,  0.34625066, -0.06388113,
         0.09097357,  0.08941821,  0.19619892,  0.43316871,  0.13188598,
         0.05029044,  0.39613396, -0.69888927, -1.42127346,  1.81054759,
         0.68297641, -4.45356885, -0.77993498,  0.85871501, -4.83122871]])

In [15]:
dt = gs_dt.best_estimator_.named_steps['decisiontreeclassifier']
pd.DataFrame(list(zip(df.columns, np.transpose(dt.feature_importances_))),
            columns=['feature','importance']).sort_values('importance',axis=0,ascending=False).head()

,feature,importance
39,38,0.708588
36,35,0.150753
28,27,0.103772
20,19,0.036887
4,3,0.000000


In [16]:
print('logistic regression(train):' ,accuracy_score(y_train, y_pred_lr_train))
print('logistic regression(test):' ,accuracy_score(y_test, y_pred_lr_test))

print('\ndecision tree(train):' ,accuracy_score(y_train, y_pred_dt_train))
print('decision tree(test):' ,accuracy_score(y_test, y_pred_dt_test))

logistic regression(train): 0.802732919255
logistic regression(test): 0.809275362319

decision tree(train): 0.802732919255
decision tree(test): 0.813333333333


In [17]:
from sklearn.metrics import confusion_matrix,classification_report
#confusion_matrix(y_test, y_pred_rf_test)
print('logistic regression\n', classification_report(y_test, y_pred_lr_test))
print('decision tree\n', classification_report(y_test, y_pred_dt_test))

logistic regression
              precision    recall  f1-score   support

          0       0.88      0.75      0.81       944
          1       0.75      0.88      0.81       781

avg / total       0.82      0.81      0.81      1725

decision tree
              precision    recall  f1-score   support

          0       0.85      0.80      0.82       944
          1       0.78      0.83      0.80       781

avg / total       0.82      0.81      0.81      1725



In [18]:
# # ==========================================================================
# pipe_rf = make_pipeline(RandomForestClassifier(random_state=1))

# param_grid_rf = [{'randomforestclassifier__n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
#                'randomforestclassifier__max_depth': [1, 2, 3, 4, 5, 6, 7, None], 
#                'randomforestclassifier__min_samples_leaf': [1, 2, 3, 4, 5, 6, 7]}]

# gs_rf = GridSearchCV(estimator=pipe_rf,
#                   param_grid=param_grid_rf,
#                   scoring='recall',
#                   cv=10,
#                   refit=True)

# gs_rf = gs_rf.fit(X_train, y_train)

# y_pred_rf_train = gs_rf.predict(X_train)
# y_pred_rf_test = gs_rf.predict(X_test)

# print('\nrandom forest(train):' ,accuracy_score(y_train, y_pred_rf_train))
# print('random forest(test):' ,accuracy_score(y_test, y_pred_rf_test))